<a href="https://colab.research.google.com/github/mmfara/Adversarial-Debiasing-Enhanced/blob/main/MPA_Compas_AL_tested_race_gender_separate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
!pip install aif360
!pip install aif360[LawSchoolGPA]
!pip install aif360[Reductions]
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
!pip install BlackBoxAuditing

pip install 'aif360[inFairness]'


In [2]:
# Import necessary libraries
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.metrics import ClassificationMetric

#STARTING OFF

In [3]:
data = pd.read_csv("/content/COMPAS_race_gender_testing.csv")

In [4]:
data.head()

,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count
0,1,1,0,0,1,0,3,1,0,0,0
1,1,1,1,0,1,4,4,1,0,0,1
2,1,0,0,1,1,14,6,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,1,0,4,0,0,0,0


In [5]:
X = data.drop(['two_year_recid'], axis =1)
y = data[['two_year_recid']]

In [6]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle =True, test_size=0.3, random_state=42)

In [7]:
print(X_train.shape)
print(X_test.shape)

(3694, 10)
(1584, 10)


In [8]:
# Create training DataFrame
train_df = X_train.copy()
train_df['two_year_recid'] = y_train

test_df = X_test.copy()
test_df['two_year_recid'] = y_test

In [9]:
# Group by race and gender, then label
group_label_counts = train_df.groupby(['race', 'sex', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts)


          Negative (0)  Positive (1)
race sex                            
0    0             212           114
     1             681           459
1    0             250           141
     1             813          1024


In [10]:
# Check value counts by group and label
group_label_counts_test = test_df.groupby(['race', 'sex', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts_test.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts_test)


          Negative (0)  Positive (1)
race sex                            
0    0             100            56
     1             288           193
1    0              96            62
     1             355           434


In [11]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Define target and protected attribute columns
target_col = 'two_year_recid'
protected_cols = ['race', 'sex']  # Intersectional attributes

# Add label back to training data temporarily
X_train_temp = X_train.copy()
X_train_temp[target_col] = y_train

# Create a composite group identifier
X_train_temp['group'] = X_train_temp[protected_cols].astype(str).agg('_'.join, axis=1)

# Pick a sampler
sampler = RandomOverSampler(random_state=42)
# sampler = SMOTE(random_state=42)

resampled_X = []
resampled_y = []

# Oversample within each intersectional group
for group in X_train_temp['group'].unique():
    group_data = X_train_temp[X_train_temp['group'] == group]

    Xg = group_data.drop(columns=[target_col])
    yg = group_data[target_col]

    # Drop group before resampling
    Xg_no_group = Xg.drop(columns=['group'], errors='ignore')

    if yg.nunique() < 2:
        print(f"Skipping group {group}: only one class present.")
        continue

    Xg_res, yg_res = sampler.fit_resample(Xg_no_group, yg)

    # Restore the group column after resampling
    Xg_res['group'] = group
    resampled_X.append(Xg_res)
    resampled_y.append(pd.Series(yg_res))

# Combine everything
X_train_bal = pd.concat(resampled_X).reset_index(drop=True)
y_train_bal = pd.concat(resampled_y).reset_index(drop=True)

# Reattach target label
X_train_bal[target_col] = y_train_bal

# ✅ View final balanced counts per intersectional group
print("✅ Final balanced class counts per (race, sex):")
X_train_bal[['group', target_col]].groupby(['group', target_col]).size().unstack(fill_value=0)


✅ Final balanced class counts per (race, sex):


two_year_recid,0,1
group,,
0_0,212,212
0_1,681,681
1_0,250,250
1_1,1024,1024


In [12]:
X_train_bal.drop(columns=['group'], inplace=True)

In [13]:
dataset_train = X_train_bal.reset_index(drop=True)
dataset_test = test_df.reset_index(drop=True)

In [14]:
dataset_train.shape

(4334, 11)

In [15]:
dataset_test.shape

(1584, 11)

Checking my datasets

In [16]:
dataset_train

,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,juv_fel_count,juv_misd_count,juv_other_count,two_year_recid
0,0,0,1,1,1,0,5,0,0,0,0
1,1,0,0,0,1,2,1,0,0,0,0
2,1,0,0,2,1,10,9,0,0,0,0
3,1,0,2,0,1,0,1,0,0,0,0
4,1,0,1,2,1,0,8,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
4329,1,1,0,2,0,3,9,0,0,0,1
4330,1,1,0,1,0,1,6,0,0,0,1
4331,1,1,0,2,0,14,10,0,0,0,1
4332,1,1,0,0,0,2,3,0,0,0,1


In [17]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

privileged_groups = [{'race': 1, 'sex': 1}]

unprivileged_groups = [
    {'race': r, 'sex': s}
    for r in data['race'].unique()
    for s in data['sex'].unique()
    if not (r == 1 and s == 1)]


#DisparateImpactRemover

In [18]:
import numpy as np
import logging
from itertools import product
from aif360.algorithms import Transformer


class DisparateImpactRemover(Transformer):
    """
    Extended Disparate Impact Remover:
    - Uses GeneralRepairer for rank computation (faithful to Feldman et al.)
    - Supports intersectional groups
    - Repairs feature values toward a shared global distribution
    - Preserves within-group rank
    """

    def __init__(self, repair_level=1.0, sensitive_attribute=None, min_group_size=20, verbose=True):
        super().__init__()
        from BlackBoxAuditing.repairers.GeneralRepairer import Repairer
        self.Repairer = Repairer

        self.repair_level = repair_level
        self.min_group_size = min_group_size
        self.verbose = verbose

        if not 0.0 <= repair_level <= 1.0:
            raise ValueError("'repair_level' must be between 0.0 and 1.0.")

        if isinstance(sensitive_attribute, str):
            self.sensitive_attributes = [sensitive_attribute]
        elif isinstance(sensitive_attribute, list):
            self.sensitive_attributes = sensitive_attribute
        elif sensitive_attribute is None:
            self.sensitive_attributes = []
        else:
            raise TypeError("sensitive_attribute must be str, list, or None")

        if self.verbose:
            logging.basicConfig(level=logging.INFO)

    def fit_transform(self, dataset):
        if not self.sensitive_attributes:
            self.sensitive_attributes = dataset.protected_attribute_names[:1]

        indices = [dataset.feature_names.index(attr) for attr in self.sensitive_attributes]
        protected_values = [np.unique(dataset.features[:, idx]) for idx in indices]
        group_combinations = list(product(*protected_values))

        repaired = dataset.copy()
        all_features = repaired.features.copy()

        feature_dim = all_features.shape[1]
        protected_set = set(indices)
        non_protected_indices = [i for i in range(feature_dim) if i not in protected_set]

        # Step 1: Collect data for global target distribution
        pooled_values = []
        group_indices = {}

        for group_vals in group_combinations:
            mask = np.logical_and.reduce([
                all_features[:, idx] == val for idx, val in zip(indices, group_vals)
            ])
            group_features = all_features[mask][:, non_protected_indices]

            if group_features.shape[0] < self.min_group_size:
                if self.verbose:
                    logging.warning(f"Skipping group {group_vals} (size={group_features.shape[0]})")
                continue

            pooled_values.append(group_features)
            group_indices[group_vals] = np.where(mask)[0]

        if not pooled_values:
            raise ValueError("No eligible groups met the minimum size requirement.")

        # Build global sorted target distribution
        pooled_array = np.vstack(pooled_values)
        global_sorted = np.sort(pooled_array, axis=0)

        # Step 2: Repair each group by aligning its ranks to global distribution
        for group_vals, idxs in group_indices.items():
            group_data = all_features[idxs][:, non_protected_indices]
            n = len(idxs)

            # Use GeneralRepairer to compute feature-wise ranks
            repairer = self.Repairer(group_data.tolist(), 0, 1.0, False)
            ranked = np.array(repairer.repair(group_data.tolist()))

            # Use the ranks to align to global distribution
            ranks = np.argsort(np.argsort(group_data, axis=0))
            aligned_by_rank = np.zeros_like(group_data)

            for col in range(group_data.shape[1]):
                aligned_by_rank[:, col] = global_sorted[ranks[:, col], col]

            # Interpolate: original + global target
            group_repaired = (
                (1 - self.repair_level) * group_data +
                self.repair_level * aligned_by_rank
            )

            all_features[idxs[:, None], non_protected_indices] = group_repaired

            if self.verbose:
                logging.info(f"Repaired group {group_vals} (size={n}) toward global target.")

        # Restore protected attributes
        for idx in indices:
            all_features[:, idx] = dataset.features[:, idx]

        repaired.features = all_features
        return repaired


In [19]:
# Applying DisparateImpactRemover
dir = DisparateImpactRemover(
    repair_level=0.4,
    sensitive_attribute=['race', 'sex'],
    min_group_size=100,
    verbose=True
)

In [20]:
dataset_train_bld = BinaryLabelDataset(df=dataset_train,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=['race', 'sex'],
                             privileged_protected_attributes=privileged_groups)


dataset_test_bld = BinaryLabelDataset(df=dataset_test,
                                      label_names=['two_year_recid'],
                                      favorable_label=favorable_label,
                                      unfavorable_label=unfavorable_label,
                                      protected_attribute_names=['race', 'sex'],
                                      privileged_protected_attributes=privileged_groups)

###Fitting disparate impact removal on the dataset

In [21]:
# Fit and transform the training data
dir_dataset = dir.fit_transform(dataset_train_bld)

In [22]:
#Disparate Impact Remover transforms dataset
dir_dataset

               instance weights        features                              \
                                                protected attribute           
                                c_charge_degree                race age_cat   
instance names                                                                
0                           1.0             0.0                 0.0     0.6   
1                           1.0             0.6                 0.0     0.0   
2                           1.0             0.6                 0.0     0.0   
3                           1.0             0.6                 0.0     1.2   
4                           1.0             0.6                 0.0     0.6   
...                         ...             ...                 ...     ...   
4329                        1.0             0.6                 1.0     0.0   
4330                        1.0             0.6                 1.0     0.0   
4331                        1.0             0.6     

#AL

In [23]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [24]:
import numpy as np
import tensorflow.compat.v1 as tf
import logging
from aif360.algorithms import Transformer
from aif360.metrics import ClassificationMetric

tf.disable_v2_behavior()

class AdversarialDebiasing(Transformer):
    def __init__(self,
                 unprivileged_groups,
                 privileged_groups,
                 scope_name,
                 sess,
                 seed=None,
                 adversary_loss_weight=0.01,  # ✅ less pressure
                 num_epochs=50,
                 batch_size=128,
                 classifier_num_hidden_units=200,
                 dropout=0.5,  # ✅ more generalization
                 early_stopping_patience=5,
                 validation_dataset=None,
                 verbose=True,
                 debias=True):
        super().__init__(
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)

        self.scope_name = scope_name
        self.seed = seed
        self.sess = sess
        self.adversary_loss_weight = adversary_loss_weight
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.classifier_num_hidden_units = classifier_num_hidden_units
        self.dropout = dropout
        self.early_stopping_patience = early_stopping_patience
        self.validation_dataset = validation_dataset
        self.verbose = verbose
        self.debias = debias

        if self.verbose:
            logging.basicConfig(level=logging.INFO)

        if self.seed is not None:
            np.random.seed(self.seed)
        self.seed1, self.seed2, self.seed3, self.seed4 = np.random.randint(1, 9999, 4)

    def _encode_protected_attributes(self, dataset):
        indices = [dataset.feature_names.index(attr) for attr in dataset.protected_attribute_names]
        values = dataset.features[:, indices]
        tuples = [tuple(row) for row in values]
        unique_combos = sorted(set(tuples))
        self.combo_to_class = {combo: i for i, combo in enumerate(unique_combos)}
        encoded = np.array([self.combo_to_class[t] for t in tuples], dtype=np.int32)
        return encoded, len(unique_combos)

    def _classifier_model(self, features, features_dim, keep_prob):
        with tf.variable_scope("classifier_model"):
            W1 = tf.get_variable('W1', [features_dim, self.classifier_num_hidden_units],
                                 initializer=tf.initializers.glorot_uniform(seed=self.seed1))
            b1 = tf.Variable(tf.zeros([self.classifier_num_hidden_units]))
            h1 = tf.nn.relu(tf.matmul(features, W1) + b1)
            h1 = tf.nn.dropout(h1, rate=1 - keep_prob, seed=self.seed2)

            W2 = tf.get_variable('W2', [self.classifier_num_hidden_units, 1],
                                 initializer=tf.initializers.glorot_uniform(seed=self.seed3))
            b2 = tf.Variable(tf.zeros([1]))
            logits = tf.matmul(h1, W2) + b2
            pred = tf.sigmoid(logits)
        return pred, logits

    def _adversary_model(self, pred_logits, true_labels, num_classes):
        with tf.variable_scope("adversary_model"):
            s = tf.sigmoid(pred_logits)
            input_concat = tf.concat([s, s * true_labels, s * (1.0 - true_labels)], axis=1)
            W = tf.get_variable('W_adv', [input_concat.shape[1], num_classes],
                                initializer=tf.initializers.glorot_uniform(seed=self.seed4))
            b = tf.Variable(tf.zeros([num_classes]))
            logits = tf.matmul(input_concat, W) + b
            preds = tf.nn.softmax(logits)
        return preds, logits

    def fit(self, dataset):
        if tf.executing_eagerly():
            raise RuntimeError("AdversarialDebiasing does not work in eager execution mode.")

        temp_labels = dataset.labels.copy()
        temp_labels[(dataset.labels == dataset.favorable_label).ravel(), 0] = 1.0
        temp_labels[(dataset.labels == dataset.unfavorable_label).ravel(), 0] = 0.0

        prot_attr_encoded, num_classes = self._encode_protected_attributes(dataset)
        num_samples, self.features_dim = dataset.features.shape
        best_val_loss = float('inf')
        patience_counter = 0

        with tf.variable_scope(self.scope_name):
            self.features_ph = tf.placeholder(tf.float32, [None, self.features_dim])
            self.true_labels_ph = tf.placeholder(tf.float32, [None, 1])
            self.protected_ph = tf.placeholder(tf.int32, [None])
            self.keep_prob = tf.placeholder(tf.float32)

            self.pred_labels, pred_logits = self._classifier_model(self.features_ph, self.features_dim, self.keep_prob)
            pred_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.true_labels_ph, logits=pred_logits))

            if self.debias:
                adv_preds, adv_logits = self._adversary_model(pred_logits, self.true_labels_ph, num_classes)
                adv_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.protected_ph, logits=adv_logits))

            classifier_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=f"{self.scope_name}/classifier_model")
            if self.debias:
                adversary_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=f"{self.scope_name}/adversary_model")
                adversary_opt = tf.train.AdamOptimizer(0.001)
                adv_grads = {v: g for g, v in adversary_opt.compute_gradients(adv_loss, classifier_vars)}
                normalize = lambda x: x / (tf.norm(x) + 1e-8)

            classifier_opt = tf.train.AdamOptimizer(0.001)
            classifier_grads = []
            for grad, var in classifier_opt.compute_gradients(pred_loss, classifier_vars):
                if self.debias:
                    u = normalize(adv_grads[var])
                    grad = grad - tf.reduce_sum(grad * u) * u - self.adversary_loss_weight * adv_grads[var]
                classifier_grads.append((tf.clip_by_value(grad, -5.0, 5.0), var))

            classifier_step = classifier_opt.apply_gradients(classifier_grads)
            if self.debias:
                with tf.control_dependencies([classifier_step]):
                    adversary_step = adversary_opt.minimize(adv_loss, var_list=adversary_vars)

            self.sess.run(tf.global_variables_initializer())

            for epoch in range(self.num_epochs):
                epoch_loss = 0
                shuffled = np.random.permutation(num_samples)

                for i in range(0, num_samples, self.batch_size):
                    batch = shuffled[i:i + self.batch_size]
                    feed = {
                        self.features_ph: dataset.features[batch],
                        self.true_labels_ph: temp_labels[batch].reshape(-1, 1),
                        self.protected_ph: prot_attr_encoded[batch],
                        self.keep_prob: self.dropout
                    }
                    if self.debias:
                        _, _, l_cls, l_adv = self.sess.run([classifier_step, adversary_step, pred_loss, adv_loss], feed_dict=feed)
                        epoch_loss += l_cls + l_adv
                    else:
                        _, l_cls = self.sess.run([classifier_step, pred_loss], feed_dict=feed)
                        epoch_loss += l_cls

                if self.verbose:
                    logging.info(f"Epoch {epoch}: Train Loss = {epoch_loss:.4f}")

                if self.validation_dataset:
                    val_loss = self._evaluate_loss(self.validation_dataset)
                    if self.verbose:
                        logging.info(f"Validation Loss = {val_loss:.4f}")
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        patience_counter = 0
                    else:
                        patience_counter += 1
                        if patience_counter >= self.early_stopping_patience:
                            logging.info("Early stopping triggered.")
                            break

        return self

    def _evaluate_loss(self, val_dataset):
        temp_labels = val_dataset.labels.copy()
        temp_labels[(val_dataset.labels == val_dataset.favorable_label).ravel(), 0] = 1.0
        temp_labels[(val_dataset.labels == val_dataset.unfavorable_label).ravel(), 0] = 0.0

        prot_attr_encoded, _ = self._encode_protected_attributes(val_dataset)

        feed = {
            self.features_ph: val_dataset.features,
            self.true_labels_ph: temp_labels,
            self.protected_ph: prot_attr_encoded,
            self.keep_prob: 1.0
        }

        preds = self.sess.run(self.pred_labels, feed_dict=feed)
        return np.mean(np.square(preds - temp_labels))

    def predict(self, dataset):
        temp_labels = dataset.labels.copy()
        temp_labels[(dataset.labels == dataset.favorable_label).ravel(), 0] = 1.0
        temp_labels[(dataset.labels == dataset.unfavorable_label).ravel(), 0] = 0.0

        num_samples = len(dataset.features)
        pred_labels = []
        for i in range(0, num_samples, self.batch_size):
            batch_features = dataset.features[i:i + self.batch_size]
            batch_labels = np.reshape(temp_labels[i:i + self.batch_size], [-1, 1])

            feed_dict = {
                self.features_ph: batch_features,
                self.true_labels_ph: batch_labels,
                self.keep_prob: 1.0
            }

            pred_labels += self.sess.run(self.pred_labels, feed_dict=feed_dict)[:, 0].tolist()

        dataset_new = dataset.copy(deepcopy=True)
        dataset_new.scores = np.array(pred_labels, dtype=np.float64).reshape(-1, 1)
        dataset_new.labels = (np.array(pred_labels) > 0.5).astype(np.float64).reshape(-1, 1)

        dataset_new.labels[(dataset_new.labels == 1.0).ravel(), 0] = dataset.favorable_label
        dataset_new.labels[(dataset_new.labels == 0.0).ravel(), 0] = dataset.unfavorable_label

        return dataset_new


Instructions for updating:
non-resource variables are not supported in the long term


In [25]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()

# Define intersectional privileged and unprivileged groups
privileged_groups = [{'race': 1, 'sex': 1}]  # e.g., White Male

# Create all other combinations except the privileged one
races = data['race'].unique()
sexes = data['sex'].unique()
unprivileged_groups = [
    {'race': r, 'sex': s}
    for r in races
    for s in sexes
    if not (r == 1 and s == 1)
]

# Initialize adversarial debiasing
adv = AdversarialDebiasing(
    privileged_groups=[{'race': 1, 'sex': 1}],
    unprivileged_groups=[
        {'race': r, 'sex': s}
        for r in data['race'].unique()
        for s in data['sex'].unique()
        if not (r == 1 and s == 1)
    ],
    scope_name='adv_debias_race_sex7',
    sess=tf.Session(),
    debias=True,
    seed=101,
    num_epochs=50,
    batch_size=128,
    classifier_num_hidden_units=100,
    dropout=0.5,
    adversary_loss_weight=0.05,  # lower pressure to prevent collapse
    early_stopping_patience=5,
    verbose=True
)




# Fit on a BinaryLabelDataset that includes 'race' and 'sex' as protected attributes
adv.fit(dir_dataset)


In [26]:
# Predict with the debias_model
predictions = adv.predict(dataset_test_bld)

In [27]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test_bld.feature_names.index('race')
sex_idx = dataset_test_bld.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test_bld.features[:, race_idx])
sex_vals = np.unique(dataset_test_bld.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test_bld,
        predictions,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test_bld.features[:, race_idx] == race_val) &
        (dataset_test_bld.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test_bld.labels[mask]
    y_pred = predictions.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.168007,-0.088134,0.746058,-0.107606,0.673077
1,0,1,-0.162636,-0.157064,0.754176,-0.078717,0.632017
2,1,0,-0.091977,-0.094470,0.860978,0.001144,0.607595


In [28]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [29]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.662879
Disparate Impact: 0.773809
Mean Difference: -0.149647


In [30]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [31]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.14964727264031374
Disparate Impact: 0.7738090074459626
Equal Opportunity Difference: -0.1321736038051773


In [32]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.132174
Difference in False Positive Rates (Unprivileged - Privileged) = -0.068845


#DIR+AL+EO

In [33]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Initialize EqualizedOdds
eo = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, seed = 101)

# Fit the EqualizedOdds model
test_predictions = eo.fit_predict(dataset_test_bld, predictions)

In [34]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [35]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.633207
Disparate Impact: 0.915937
Mean Difference: -0.046986


In [36]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [37]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.04698567568213885
Disparate Impact: 0.9159371924870577
Equal Opportunity Difference: 0.015291833982841174


In [38]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = 0.015292
Difference in False Positive Rates (Unprivileged - Privileged) = -0.020958


In [41]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test_bld.feature_names.index('race')
sex_idx = dataset_test_bld.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test_bld.features[:, race_idx])
sex_vals = np.unique(dataset_test_bld.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test_bld.features[:, race_idx] == race_val) &
        (dataset_test_bld.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test_bld.labels[mask]
    y_pred = test_predictions.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.065346,0.059332,0.883089,-0.059718,0.673077
1,0,1,-0.059975,-0.009599,0.892698,-0.030829,0.632017
2,1,0,0.010685,0.052995,1.019117,0.049032,0.607595


#DIR+AL+ROC

In [42]:
# Apply Reject Option Classification
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, metric_name="Average odds difference",
                                 low_class_thresh=0.3, high_class_thresh=0.8, num_class_thresh=100, num_ROC_margin=50,
                                 metric_ub=0.05, metric_lb=-0.05)

In [43]:
test_predictions = ROC.fit_predict(dataset_test_bld, predictions)

In [44]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [45]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.683712
Disparate Impact: 0.815229
Mean Difference: -0.091800


In [46]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [47]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.09179998565176839
Disparate Impact: 0.8152291105121294
Equal Opportunity Difference: -0.07704446782343266


In [48]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.077044
Difference in False Positive Rates (Unprivileged - Privileged) = 0.002118


In [49]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test_bld.feature_names.index('race')
sex_idx = dataset_test_bld.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test_bld.features[:, race_idx])
sex_vals = np.unique(dataset_test_bld.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test_bld.features[:, race_idx] == race_val) &
        (dataset_test_bld.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test_bld.labels[mask]
    y_pred = test_predictions.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.099396,0.003456,0.799941,-0.038873,0.730769
1,0,1,-0.097663,-0.105167,0.803428,0.005849,0.656965
2,1,0,-0.066452,-0.062212,0.866249,0.033627,0.658228
